In [1]:
import sys
!{sys.executable} -m pip install qdrant-client


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv(os.path.join(os.getcwd(),'input/mfi_embeddings_title.csv'))
df.head()

,Adresse,Inhaltstyp,Status-Code,Status,Indexierbarkeit,Indexierbarkeitsstatus,Extract embeddings from page content,Crawlzeitstempel
0,https://www.meggle-foodingredients.de/,text/html; charset=UTF-8,200,OK,Indexierbar,NaN,"-0.14715327322483063,-0.2957857847213745,0.068...",2025-07-22 21:46:20
1,https://www.meggle-foodingredients.de/wp-sitem...,application/xml; charset=UTF-8,200,OK,Indexierbar,NaN,NaN,2025-07-22 21:46:20
2,https://www.meggle-foodingredients.de/savoury/,text/html; charset=UTF-8,200,OK,Indexierbar,NaN,"-0.2507178485393524,-0.2408612072467804,-0.249...",2025-07-22 21:46:20
3,https://www.meggle-foodingredients.de/produkt-...,text/html; charset=UTF-8,200,OK,Indexierbar,NaN,"-0.34509339928627014,-0.06488354504108429,-0.0...",2025-07-22 21:46:20
4,https://www.meggle-foodingredients.de/news/von...,text/html; charset=UTF-8,200,OK,Indexierbar,NaN,"-0.38861164450645447,-0.07005718350410461,0.33...",2025-07-22 21:46:20


In [3]:
df_qrs = df[['Adresse', 'Extract embeddings from page content']]
df_qrs = df_qrs.rename(columns={'Extract embeddings from page content': 'embeddings'})
df_qrs['Adresse'] = df_qrs['Adresse'].astype(str)
df_qrs.dropna(subset=['embeddings'], inplace=True)

# for i, row in df_qrs.iterrows():
#     # Convert the embeddings string to a list of floats
#     embeddings_list = row['embeddings'].split(',')
#     df_qrs.at[i, 'embeddings'] = [float(x) for x in embeddings_list]
# cols = [1,2,3,4,5,6,8]
# df_qrs.drop(df.columns[cols],axis=1,inplace=True)
df_qrs.head()

,Adresse,embeddings
0,https://www.meggle-foodingredients.de/,"-0.14715327322483063,-0.2957857847213745,0.068..."
2,https://www.meggle-foodingredients.de/savoury/,"-0.2507178485393524,-0.2408612072467804,-0.249..."
3,https://www.meggle-foodingredients.de/produkt-...,"-0.34509339928627014,-0.06488354504108429,-0.0..."
4,https://www.meggle-foodingredients.de/news/von...,"-0.38861164450645447,-0.07005718350410461,0.33..."
5,https://www.meggle-foodingredients.de/dairy/,"-0.23975729942321777,0.047614485025405884,0.51..."


In [4]:
for i, row in df_qrs.iterrows():
    # Convert the embeddings string to a list of floats
    embeddings_list = row['embeddings'].split(',')
    df_qrs.at[i, 'embeddings'] = [float(x) for x in embeddings_list]

df_qrs.head()

,Adresse,embeddings
0,https://www.meggle-foodingredients.de/,"[-0.14715327322483063, -0.2957857847213745, 0...."
2,https://www.meggle-foodingredients.de/savoury/,"[-0.2507178485393524, -0.2408612072467804, -0...."
3,https://www.meggle-foodingredients.de/produkt-...,"[-0.34509339928627014, -0.06488354504108429, -..."
4,https://www.meggle-foodingredients.de/news/von...,"[-0.38861164450645447, -0.07005718350410461, 0..."
5,https://www.meggle-foodingredients.de/dairy/,"[-0.23975729942321777, 0.047614485025405884, 0..."


In [5]:
import ollama

model = 'mxbai-embed-large'

In [6]:
def generate_embeddings(text, model=model):
    return ollama.embeddings(model=model, prompt=text)["embedding"]

In [7]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
from ast import literal_eval

client = QdrantClient(":memory:")
# client = QdrantClient(url="http://localhost:6333")

# Convert the embeddings column to a list of floats
vector_size = len(df_qrs["embeddings"][0])

# Check if the collection already exists
# and create it if it doesn't
if not client.collection_exists("test_collection"):
   client.create_collection(
      collection_name="test_collection",
      vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
   )

# Upsert points into the collection
operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(
            id=k,
            vector=v["embeddings"],
            payload={"urls": v["Adresse"]},
        )
        for k, v in df_qrs.iterrows()
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [9]:
limit = 1
def perform_search(query, limit=limit):
    return client.query_points(
        collection_name="test_collection",
        query=query,
        with_vectors=True,
        with_payload=["urls"],
        limit=limit,
    ).points

# Which of our stored vectors are most similar to the query vector?
# search_result = client.query_points(
#     collection_name="test_collection",
#     query=query,
#     with_vectors=True,
#     with_payload=["urls"],
#     limit=limit,
#     # score_threshold=0.1
# ).points

# for i in search_result:
#     print(f'{i.score}: {i.payload["urls"]}')

In [18]:
search_term = "buttermilchpulver"

search_result = perform_search(
    query=generate_embeddings(search_term),
    limit=limit,
)

prompt=f"Welche der folgenden URLs {search_result[0].payload['urls']} beschreibt das Thema {search_term} am besten?"
# prompt=f"Which of the following urls describes this topic {search_term} best: {search_result[0].payload['urls']}"

# model = 'stablelm2'
model = 'mistral'
# model = 'avr/sfr-embedding-mistral'

output = ollama.generate(
    model=model,
    prompt=prompt,
)
print(output['response'])

 Die URL <https://www.meggle-foodingredients.de/buttermilchpulver/> beschreibt Buttermilchpulver am besten, denn sie führt zu einer Seite von Meggle Foodingredients, die speziell auf das Thema Buttermilchpulver fokussiert ist und Informationen zu diesem Produkt bietet.
